In [1]:
#!pip install langchain langchain-google-genai sqlalchemy pymysql dotenv langchain_community

In [2]:
import os
from operator import itemgetter
from sqlalchemy import create_engine
from dotenv import load_dotenv, find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_classic.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

/home/temirlan/workstation/python/db-final/venv/lib64/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [3]:
load_dotenv()

True

In [4]:
api_key = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = api_key

In [5]:
DB_URI = os.getenv("CONNECTION_STRING")
engine = create_engine(DB_URI)

In [6]:
db = SQLDatabase.from_uri(DB_URI)
llm = ChatGoogleGenerativeAI(model="gemini-flash-latest", temperature=0.2)

sql_chain = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

/tmp/ipykernel_29542/1557315322.py:5: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the `langchain-community package and should be used instead. To use it run `pip install -U `langchain-community` and import as `from `langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query = QuerySQLDataBaseTool(db=db)


In [7]:
def extract_sql(text):
    if "SQLQuery:" in text:
        return text.split("SQLQuery:")[-1].strip()
    return text.strip()

In [8]:
answer_prompt = ChatPromptTemplate.from_template(
"""You are a helpful assistant that answers questions about a database.
    
Given the following user question, corresponding SQL query, and SQL result, answer the user question in a natural way.

Question: {question}
SQL Query: {query}
SQL Result: {result}

Answer: """
)

chain = (
    RunnablePassthrough
        .assign(query=sql_chain)
        .assign(query=lambda x: extract_sql(x["query"]))
        .assign(result=lambda x: execute_query.invoke(x["query"]))
    | answer_prompt
    | llm
    | StrOutputParser()
)

In [9]:
question = \
"""
Describe the DB please
"""

response = chain.invoke({"question": question})
print(f"{response}")

ChatGoogleGenerativeAIError: Error calling model 'gemini-flash-latest' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 48.325253146s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectP